# How to Debug Your New Theory

Some helpful strategies:

1`.` Get better at programming

 2`.` Check your sanity

3`.` Pick an easier problem

  ## 1. Cleaner code

Strategy:

 - Don't reinvent wheels — look for existing libraries

 - If you write the same 2+ lines of code twice, think about packaging it into a module

 - The more often you use a module, the more carefully designed it needs to be

 - Fix design problems early and often

 - Write **comprehensive** unit tests!

### The `scfexchange` module

In [7]:
import numpy as np
from scfexchange import Nuclei

labels = ("O", "H", "H")
coords = np.array([[0.,  0.,  0.],
                   [0., -1., -1.],
                   [0., +1., -1.]])
nuclei = Nuclei(labels, coords)

In [8]:
nuclei

Nuclei(
O     0.0000000000    0.0000000000    0.0000000000
H     0.0000000000   -1.0000000000   -1.0000000000
H     0.0000000000    1.0000000000   -1.0000000000
)

In [9]:
nuclei.get_center_of_mass()

array([ 0.        ,  0.        , -0.11191487])

In [10]:
nuclei.get_dipole_moment()

array([ 0.,  0., -2.])

In [11]:
from scfexchange import Molecule

molecule = Molecule(nuclei, charge=1, multiplicity=2)

In [12]:
molecule

Molecule(charge 1, multiplicity 2, Nuclei(
O     0.0000000000    0.0000000000    0.0000000000
H     0.0000000000   -1.0000000000   -1.0000000000
H     0.0000000000    1.0000000000   -1.0000000000
))

In [13]:
molecule.nalpha

5

In [14]:
molecule.nbeta

In [15]:
from scfexchange.psi4_interface import Integrals

integrals = Integrals(nuclei, "sto-3g")

In [16]:
import inspect

inspect.getmro(Integrals)

(scfexchange.psi4_interface.Integrals,
 scfexchange.integrals.IntegralsInterface,
 object)

In [17]:
v_ao = integrals.get_ao_1e_potential()
v_aso = integrals.get_ao_1e_potential(use_spinorbs=True)

In [18]:
print(v_ao.round(0))

[[-62.  -8.   0.   0.  -0.  -3.  -3.]
 [ -8. -10.   0.   0.   0.  -6.  -6.]
 [  0.   0. -10.   0.   0.   3.   3.]
 [  0.   0.   0. -10.   0.   0.   0.]
 [ -0.   0.   0.   0. -10.   3.  -3.]
 [ -3.  -6.   3.   0.   3.  -7.  -3.]
 [ -3.  -6.   3.   0.  -3.  -3.  -7.]]


In [19]:
print(v_aso.round(0))

[[-62.  -8.   0.   0.  -0.  -3.  -3.  -0.  -0.   0.   0.  -0.  -0.  -0.]
 [ -8. -10.   0.   0.   0.  -6.  -6.  -0.  -0.   0.   0.   0.  -0.  -0.]
 [  0.   0. -10.   0.   0.   3.   3.   0.   0.  -0.   0.   0.   0.   0.]
 [  0.   0.   0. -10.   0.   0.   0.   0.   0.   0.  -0.   0.   0.   0.]
 [ -0.   0.   0.   0. -10.   3.  -3.  -0.   0.   0.   0.  -0.   0.  -0.]
 [ -3.  -6.   3.   0.   3.  -7.  -3.  -0.  -0.   0.   0.   0.  -0.  -0.]
 [ -3.  -6.   3.   0.  -3.  -3.  -7.  -0.  -0.   0.   0.  -0.  -0.  -0.]
 [ -0.  -0.   0.   0.  -0.  -0.  -0. -62.  -8.   0.   0.  -0.  -3.  -3.]
 [ -0.  -0.   0.   0.   0.  -0.  -0.  -8. -10.   0.   0.   0.  -6.  -6.]
 [  0.   0.  -0.   0.   0.   0.   0.   0.   0. -10.   0.   0.   3.   3.]
 [  0.   0.   0.  -0.   0.   0.   0.   0.   0.   0. -10.   0.   0.   0.]
 [ -0.   0.   0.   0.  -0.   0.  -0.  -0.   0.   0.   0. -10.   3.  -3.]
 [ -0.  -0.   0.   0.   0.  -0.  -0.  -3.  -6.   3.   0.   3.  -7.  -3.]
 [ -0.  -0.   0.   0.  -0.  -0.  -0.  -3.  -6.   3.

In [20]:
from scfexchange.psi4_interface import Orbitals

orbitals = Orbitals(integrals, charge=0, multiplicity=1)

In [21]:
inspect.getmro(Orbitals)

(scfexchange.psi4_interface.Orbitals,
 scfexchange.orbitals.OrbitalsInterface,
 object)

In [22]:
orbitals.get_energy()

11.813708498984759

In [23]:
nuclei.get_energy()

11.813708498984759

In [24]:
orbitals.solve()
orbitals.get_energy()

-74.735195833159139

In [25]:
orbitals.get_mo_1e_kinetic(mo_block='v,o', spin_sector='a').round(1)

array([[ 4.6,  1.8,  0. , -0.2,  0. ],
       [-0. , -0. ,  0.9,  0. , -0. ]])

In [26]:
orbitals.get_mo_1e_kinetic(mo_block='v,o', spin_sector='s').round(1)

array([[ 4.6,  0. ,  1.8,  0. ,  0. ,  0. , -0.2,  0. ,  0. ,  0. ],
       [ 0. ,  4.6,  0. ,  1.8,  0. ,  0. ,  0. , -0.2,  0. ,  0. ],
       [-0. ,  0. , -0. ,  0. ,  0.9,  0. ,  0. ,  0. , -0. ,  0. ],
       [ 0. , -0. ,  0. , -0. ,  0. ,  0.9,  0. ,  0. ,  0. , -0. ]])

In [27]:
orbitals.get_mo_1e_core_hamiltonian(spin_sector='a').round(0)

array([[-33.,  -1.,   0.,  -0.,   0.,  -0.,  -0.],
       [ -1.,  -8.,  -0.,  -0.,   0.,  -2.,   0.],
       [  0.,  -0.,  -7.,   0.,   0.,  -0.,  -2.],
       [ -0.,  -0.,   0.,  -7.,  -0.,   1.,  -0.],
       [  0.,   0.,   0.,  -0.,  -8.,  -0.,   0.],
       [ -0.,  -2.,  -0.,   1.,  -0.,  -5.,  -0.],
       [ -0.,   0.,  -2.,  -0.,   0.,  -0.,  -6.]])

Electric field perturbation

$$ h_p^q\mapsto h_p^q - \boldsymbol{\mathcal{E}}\cdot\boldsymbol{\mu}_p^q $$

In [30]:
orbitals.get_mo_1e_core_hamiltonian(electric_field=[0., 0., 30.], spin_sector='a').round(0)

array([[-30.,  -0.,   0.,  -2.,   0.,   1.,  -0.],
       [ -0., -20.,  -0.,  17.,  -0.,   6.,  -0.],
       [  0.,  -0., -15.,  -0.,   0.,   0.,  17.],
       [ -2.,  17.,  -0.,   5.,  -0., -12.,   0.],
       [  0.,  -0.,   0.,  -0.,  -4.,   0.,  -0.],
       [  1.,   6.,   0., -12.,   0., -33.,   0.],
       [ -0.,  -0.,  17.,   0.,  -0.,   0., -23.]])

In [31]:
goovv = orbitals.get_mo_2e_repulsion(mo_block='o,o,v,v',
                                     antisymmetrize=True)
eo = orbitals.get_mo_fock_diagonal(mo_space='o')
ev = orbitals.get_mo_fock_diagonal(mo_space='v')

$$ t_{ab}^{ij} = \overline{g}_{ab}^{ij}\ (\epsilon_i + \epsilon_j - \epsilon_a - \epsilon_b)^{-1} $$

In [32]:
x = np.newaxis
t2 = goovv / (eo[:,x,x,x] + eo[x,:,x,x]
              - ev[x,x,:,x] - ev[x,x,x,:])

$$ E_\text{corr} = \tfrac{1}{4} \overline{g}_{ij}^{ab} t_{ab}^{ij} $$

In [33]:
1. / 4 * np.sum(goovv * t2)

-0.023242220885911075

In [34]:
from scfexchange.examples.mp2 import MP2Density

density = MP2Density(orbitals)

inspect.getmro(MP2Density)

(scfexchange.examples.mp2.MP2Density,
 scfexchange.density.CorrelatedDensityInterface,
 scfexchange.density.DensityInterface,
 object)

In [35]:
density.get_energy()

-74.735195833159111

In [36]:
density.solve()
density.get_energy()

-74.75843805404503

In [37]:
density.get_energy() - density.ref.get_energy()

-0.023242220885919096

In [38]:
inspect.getmro(type(density.ref))

(scfexchange.density.DeterminantDensity,
 scfexchange.density.DensityInterface,
 object)

In [39]:
h = orbitals.get_mo_1e_core_hamiltonian()
g = orbitals.get_mo_2e_repulsion(antisymmetrize=True)
e_nuc = nuclei.get_energy()

gamma1 = density.get_1e_moment()
gamma2 = density.get_2e_moment()

$$E = h_p^q \gamma^p_q + \tfrac{1}{4} \overline{g}_{pq}^{rs}\gamma^{pq}_{rs} + E_\text{nuc}$$

In [40]:
np.sum(h * gamma1) + 1. / 4 * np.sum(g * gamma2) + e_nuc

-74.75843805404503

## 2. Sanity checks

Strategy:

 - Weak tests

> Should this matrix be symmetric? Positive definite?

> Should these eigenvalues have spin symmetry (triples, quintets, etc.)?

> Can I drop terms to get Hartree-Fock? MP2?

 - Strong tests

> Can I get the same answer a different way?

> The holy grail: numerical tests

### The Hellmann-Feynman theorem

$$ E(\mathcal{E}) = \langle \Psi | H(\mathcal{E}) | \Psi \rangle $$

$$ \frac{\partial E(\mathcal{E})}{\partial \mathcal{E}} = \langle \tfrac{\partial\Psi}{\partial \mathcal{E}} | H(\mathcal{E}) | \Psi\rangle + \langle \Psi | \frac{\partial H(\mathcal{E})}{\partial \mathcal{E}} | \Psi\rangle + \langle \Psi | H(\mathcal{E}) | \tfrac{\partial\Psi}{\partial \mathcal{E}}\rangle $$

Hellmann-Feynman theorem says that

$$ \frac{\partial E(\mathcal{E})}{\partial \mathcal{E}} = \langle \Psi | \frac{\partial H(\mathcal{E})}{\partial \mathcal{E}} | \Psi\rangle $$

for an exact $\Psi$

**or** an approximate one that is stationary!

Therefore, we can calculate the electric dipole moment as follows.

$$ H(\boldsymbol{\mathcal{E}}) = H_\text{elec} - \boldsymbol{\mu} \cdot \boldsymbol{\mathcal{E}} $$

$$ \langle \Psi|\boldsymbol{\mu}|\Psi \rangle = - \langle \Psi | \dfrac{\partial H(\boldsymbol{\mathcal{E}})}{\partial \boldsymbol{\mathcal{E}}} | \Psi\rangle = - \dfrac{\partial E(\boldsymbol{\mathcal{E}})}{\partial \boldsymbol{\mathcal{E}}} $$

$\langle\Psi|\boldsymbol{\mu}|\Psi\rangle$ can be evaluated analytically

$\dfrac{\partial E(\boldsymbol{\mathcal{E}})}{\partial \boldsymbol{\mathcal{E}}}$
can be evaluated numerically

$$ \dfrac{\partial E(\boldsymbol{\mathcal{E}})}{\partial \boldsymbol{\mathcal{E}}} = \lim_{\delta\boldsymbol{\mathcal{E}}\rightarrow0} \dfrac{E(\boldsymbol{\mathcal{E}} + \delta\boldsymbol{\mathcal{E}}) - E(\boldsymbol{\mathcal{E}} - \delta\boldsymbol{\mathcal{E}})}{2\|\delta\boldsymbol{\mathcal{E}}\|} $$

If
$ \langle\Psi|\boldsymbol{\mu}|\Psi\rangle \not= - \dfrac{\partial E(\boldsymbol{\mathcal{E}})}{\partial \boldsymbol{\mathcal{E}}} $,
your code is wrong.

If they *are* equal, it's **probably right**!

### analytic $\langle\Psi|\boldsymbol{\mu}|\Psi\rangle$

In [41]:
from scfexchange import DeterminantDensity

density = DeterminantDensity(orbitals)

In [42]:
d = orbitals.get_mo_1e_dipole()

gamma1 = density.get_1e_moment()

$$ \langle\Psi|\boldsymbol{\mu}|\Psi\rangle = \boldsymbol{\mu}_p^q \gamma^p_q $$

In [43]:
mu = np.array([np.sum(d_x * gamma1) for d_x in d])

In [44]:
mu.round(9)

array([ 0.        ,  0.        ,  0.04607383])

In [45]:
density.get_dipole_moment().round(9)

array([ 0.        ,  0.        ,  0.04607383])

### interlude: my $\boldsymbol{\mathcal{E}}$-perturbed Hartree-Fock code

In [46]:
from scfexchange.examples.phf import PerturbedHartreeFock

phf = PerturbedHartreeFock(integrals)

In [47]:
inspect.getmro(PerturbedHartreeFock)

(scfexchange.examples.phf.PerturbedHartreeFock,
 scfexchange.orbitals.OrbitalsInterface,
 object)

In [48]:
phf.solve()

E= -74.735195833159281 ( 16 iterations, dE:   0e+00, orb grad: 8.5e-09)


In [49]:
phf.solve(electric_field=[0., 0., 1.])

E= -76.254667763944724 ( 17 iterations, dE:   6e-14, orb grad: 4.6e-09)


In [50]:
phf.get_energy(electric_field=[0., 0., 1.])

-76.254667763944724

In [51]:
phf.get_energy()

-72.868301996487588

In [52]:
energy_fn = phf.get_energy_field_function(e_threshold=1e-13, d_threshold=1e-12)

In [53]:
energy_fn([0., 0., 1])

E= -76.254667763944695 (  8 iterations, dE:   0e+00, orb grad: 7.9e-13)


-76.254667763944695

In [69]:
energy_fn([0., 0., 0.])

E= -74.735195833159310 ( 20 iterations, dE:   0e+00, orb grad: 7.9e-13)


-74.73519583315931

### numerical $\dfrac{\underline{\partial E(\boldsymbol{\mathcal{E}})}}{\partial \boldsymbol{\mathcal{E}}}$

In [55]:
import numdifftools as ndt

grad_fn = ndt.Gradient(energy_fn, step=0.005)

grad = grad_fn(np.r_[0., 0., 0.])

E= -74.735195833159281 (  1 iterations, dE:   0e+00, orb grad: 9.1e-14)
E= -74.735196013366647 ( 20 iterations, dE:   3e-14, orb grad: 5.0e-13)
E= -74.735196013366618 ( 20 iterations, dE:   0e+00, orb grad: 9.9e-13)
E= -74.735224948259145 ( 20 iterations, dE:   0e+00, orb grad: 5.0e-13)
E= -74.735224948259173 ( 18 iterations, dE:   3e-14, orb grad: 3.4e-13)
E= -74.735446619284119 ( 20 iterations, dE:   0e+00, orb grad: 8.7e-13)
E= -74.734985790464435 ( 21 iterations, dE:   3e-14, orb grad: 5.4e-13)


In [56]:
grad.round(9)

array([-0.        ,  0.        , -0.04608288])

In [57]:
mu.round(9)

array([ 0.        ,  0.        ,  0.04607383])

In [58]:
grad_fn = ndt.Gradient(energy_fn, step=0.005, order=4)

grad = grad_fn(np.r_[0., 0., 0.])

E= -74.735195833159310 ( 20 iterations, dE:   3e-14, orb grad: 7.9e-13)
E= -74.735196554005682 ( 20 iterations, dE:   0e+00, orb grad: 9.9e-13)
E= -74.735196554005711 ( 21 iterations, dE:   6e-14, orb grad: 7.5e-13)
E= -74.735312294887763 ( 21 iterations, dE:   0e+00, orb grad: 4.0e-13)
E= -74.735312294887706 ( 18 iterations, dE:   6e-14, orb grad: 7.4e-13)
E= -74.735738422409440 ( 21 iterations, dE:   0e+00, orb grad: 6.9e-13)
E= -74.734816221871824 ( 21 iterations, dE:   0e+00, orb grad: 1.0e-12)
E= -74.735196013366590 ( 21 iterations, dE:   3e-14, orb grad: 5.9e-13)
E= -74.735196013366618 ( 20 iterations, dE:   3e-14, orb grad: 9.9e-13)
E= -74.735224948259173 ( 20 iterations, dE:   6e-14, orb grad: 5.0e-13)


E= -74.735224948259145 ( 18 iterations, dE:   0e+00, orb grad: 3.3e-13)
E= -74.735446619284090 ( 20 iterations, dE:   0e+00, orb grad: 8.6e-13)
E= -74.734985790464435 ( 21 iterations, dE:   3e-14, orb grad: 5.4e-13)


In [59]:
grad.round(9)

In [60]:
mu.round(9)

array([ 0.        ,  0.        ,  0.04607383])

Hellmann-Feynman works for single-point energies.

But it has consequences that affect other properties!

### Testing linear-response codes

Static dipole polarizability

$$ \boldsymbol{\alpha} = \langle\langle \boldsymbol{\mu}; \boldsymbol{\mu} \rangle\rangle_{\omega=0} = \dfrac{\partial^2 E(\boldsymbol{\mathcal{E}}) }{ \partial \boldsymbol{\mathcal{E}}\,\partial \boldsymbol{\mathcal{E}}} $$

Analytically:

$$ \langle\langle \boldsymbol{\mu}; \boldsymbol{\mu} \rangle\rangle_{\omega=0} = \boldsymbol{r}^\dagger \mathbf{T}_\mu + \mathbf{T}_\mu^\dagger\,\boldsymbol{r} + \boldsymbol{r}^\dagger \mathbf{E}\,\boldsymbol{r} $$

$$ = - \mathbf{T}_\mu^\dagger\mathbf{E}^{-1}\mathbf{T}_\mu \hspace{20pt} \text{(via H.F. thm.)} $$

### analytic $\langle\langle \boldsymbol{\mu}; \boldsymbol{\mu} \rangle\rangle_{\omega=0}$

In [61]:
from scfexchange.examples.rpa import RPA

rpa = RPA(orbitals)

In [62]:
a = rpa.get_a_matrix()
b = rpa.get_b_matrix()
d = rpa.get_dipole_gradient_matrix()
e = np.bmat([[a, b], [b, a]])
t = np.bmat([[d], [d]])

$$ \langle\langle \boldsymbol{\mu}; \boldsymbol{\mu} \rangle\rangle_{\omega=0} = - \mathbf{T}_\mu^\dagger \mathbf{E}^{-1}\mathbf{T}_\mu $$

In [63]:
import scipy.linalg as spla

alpha = - t.T.dot(spla.inv(e).dot(t))

In [64]:
alpha.round(9)

array([[-0.01441647, -0.        , -0.        ],
       [-0.        , -2.32919913, -0.        ],
       [-0.        , -0.        , -1.62972306]])

### numerical $\dfrac{\underline{\partial^2 E(\boldsymbol{\mathcal{E}})}}{\partial \boldsymbol{\mathcal{E}}\partial \boldsymbol{\mathcal{E}}}$

In [70]:
hess_fn = ndt.Hessian(energy_fn, step=0.005)

hess = hess_fn(np.r_[0., 0., 0.])

E= -74.735195833159281 (  1 iterations, dE:   0e+00, orb grad: 1.1e-13)
E= -74.735196554005711 ( 20 iterations, dE:   3e-14, orb grad: 9.9e-13)
E= -74.735196554005682 ( 21 iterations, dE:   3e-14, orb grad: 7.5e-13)
E= -74.735225128431040 ( 21 iterations, dE:   0e+00, orb grad: 5.7e-13)
E= -74.735225128431068 ( 18 iterations, dE:   0e+00, orb grad: 3.4e-13)
E= -74.735225128431068 ( 21 iterations, dE:   3e-14, orb grad: 3.8e-13)
E= -74.735225128431040 ( 18 iterations, dE:   0e+00, orb grad: 3.5e-13)
E= -74.735446798686979 ( 21 iterations, dE:   3e-14, orb grad: 5.0e-13)


E= -74.734985971416847 ( 21 iterations, dE:   6e-14, orb grad: 5.4e-13)
E= -74.735446798687008 ( 21 iterations, dE:   3e-14, orb grad: 7.4e-13)
E= -74.734985971416876 ( 21 iterations, dE:   0e+00, orb grad: 5.4e-13)
E= -74.735312294887763 ( 20 iterations, dE:   3e-14, orb grad: 9.9e-13)
E= -74.735312294887734 ( 18 iterations, dE:   3e-14, orb grad: 7.4e-13)
E= -74.735475934889820 ( 20 iterations, dE:   0e+00, orb grad: 8.8e-13)
E= -74.735014706434669 ( 21 iterations, dE:   0e+00, orb grad: 5.4e-13)
E= -74.735475934889820 ( 21 iterations, dE:   0e+00, orb grad: 6.4e-13)


E= -74.735014706434669 ( 21 iterations, dE:   3e-14, orb grad: 5.4e-13)
E= -74.735738422409440 ( 22 iterations, dE:   3e-14, orb grad: 3.8e-13)
E= -74.734816221871824 ( 21 iterations, dE:   0e+00, orb grad: 1.0e-12)


In [66]:
hess.round(9)

array([[-0.01441693,  0.        ,  0.        ],
       [ 0.        , -2.32923457,  0.        ],
       [ 0.        ,  0.        , -1.62977963]])

In [67]:
alpha.round(9)

array([[-0.01441647, -0.        , -0.        ],
       [-0.        , -2.32919913, -0.        ],
       [-0.        , -0.        , -1.62972306]])

In [68]:
hess_diag_fn = ndt.Hessdiag(energy_fn, step=0.005, order=6)

hess_diag = hess_diag_fn(np.r_[0., 0., 0.])

E= -74.735195833159310 ( 21 iterations, dE:   0e+00, orb grad: 5.7e-13)
E= -74.735197014217761 ( 21 iterations, dE:   3e-14, orb grad: 4.8e-13)
E= -74.735197014217789 ( 21 iterations, dE:   3e-14, orb grad: 9.6e-13)
E= -74.735386645908278 ( 21 iterations, dE:   0e+00, orb grad: 5.4e-13)
E= -74.735386645908306 ( 19 iterations, dE:   0e+00, orb grad: 3.3e-13)
E= -74.735919851798940 ( 21 iterations, dE:   0e+00, orb grad: 9.2e-13)
E= -74.734738843531403 ( 22 iterations, dE:   3e-14, orb grad: 4.5e-13)
E= -74.735196294495779 ( 21 iterations, dE:   3e-14, orb grad: 7.7e-13)
E= -74.735196294495779 ( 21 iterations, dE:   0e+00, orb grad: 6.0e-13)


E= -74.735270368257289 ( 20 iterations, dE:   3e-14, orb grad: 8.3e-13)
E= -74.735270368257261 ( 18 iterations, dE:   0e+00, orb grad: 5.6e-13)
E= -74.735616761430634 ( 21 iterations, dE:   3e-14, orb grad: 5.3e-13)
E= -74.734879209480653 ( 21 iterations, dE:   0e+00, orb grad: 8.3e-13)
E= -74.735196013366618 ( 21 iterations, dE:   0e+00, orb grad: 4.9e-13)
E= -74.735196013366647 ( 20 iterations, dE:   0e+00, orb grad: 9.9e-13)
E= -74.735224948259173 ( 20 iterations, dE:   0e+00, orb grad: 5.0e-13)
E= -74.735224948259173 ( 18 iterations, dE:   3e-14, orb grad: 3.3e-13)
E= -74.735446619284090 ( 20 iterations, dE:   3e-14, orb grad: 8.6e-13)


E= -74.734985790464435 ( 21 iterations, dE:   3e-14, orb grad: 5.4e-13)


In [71]:
hess_diag.round(9)

In [74]:
alpha.diagonal().round(9)

array([[-0.01441647, -2.32919913, -1.62972306]])

# $$ \boldsymbol{{\mathscr{F}}\hspace{-1pt}{\mathit{in.}}} $$